In [1]:
# SVM
# Reference: https://www.kaggle.com/code/mehmetlaudatekman/text-classification-svm-explained/notebook

In [2]:
# Drive dependencies
'''
from google.colab import drive
drive.mount('/content/drive')

import os
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/IN3045 NLP CW'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n\nimport os\nGOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'Colab Notebooks/IN3045 NLP CW'\nGOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)\nprint(os.listdir(GOOGLE_DRIVE_PATH))\n"

In [3]:
# Import modules

from joblib import dump, load
import numpy as np
import pandas as pd
from sklearn import metrics
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [4]:
# Load data

# Create paths to train and test sets
train_path = 'Data/train.csv'
test_path = 'Data/test.csv'
#train_path = '/content/drive/MyDrive/Colab Notebooks/IN3045 NLP CW/data/train.csv'
#test_path = '/content/drive/MyDrive/Colab Notebooks/IN3045 NLP CW/data/test.csv'

# Create original unmodified train and test dataframes for record
original_train_df = pd.read_csv(train_path)
original_test_df = pd.read_csv(test_path)

# Create train and test dataframes to be modified
train_df = original_train_df
test_df = original_test_df

# Define function to modify dataframe
def modify_df(df):

    # Fill na with empty string
    df = df.fillna('')

    # Set default value of numerical label to 0
    df['Label'] = 0

    # Transform string label to numerical label
    for i in range(len(df)):
        if df.loc[i, 'Category'] == 'business':
            df.loc[i, 'Label'] = 1
        elif df.loc[i, 'Category'] == 'sports':
            df.loc[i, 'Label'] = 2
        elif df.loc[i, 'Category'] == 'politics':
            df.loc[i, 'Label'] = 3
        elif df.loc[i, 'Category'] == 'health':
            df.loc[i, 'Label'] = 4
        elif df.loc[i, 'Category'] == 'entertainment':
            df.loc[i, 'Label'] = 5
        else:
            df.loc[i, 'Label'] = 6
    # Drop unnecessary columns
    df = df.drop(['Excerpt', 'Category'], axis=1)

    return df

# Modify train and test dataframes
train_df = modify_df(train_df)
test_df = modify_df(test_df)

In [5]:
# Data preprocessing

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Define function to clean title column
def clean_df_title_col(df_title_col):

    # Create variables
    clean_title_list = []
    lemma = WordNetLemmatizer()
    swords = stopwords.words("english")

    for text in df_title_col:
        # Clean links
        text = re.sub(r'http\S+', '', text)

        # Clean everything except alphabetical and numerical characters
        text = re.sub("[^a-zA-Z0-9]"," ",text)

        # Tokenise and lemmatise
        text = nltk.word_tokenize(text.lower())
        text = [lemma.lemmatize(word) for word in text]

        # Remove stopwords
        text = [word for word in text if word not in swords]

        # Join text
        text = " ".join(text)

        # Append text to list of clean titles
        clean_title_list.append(text)

    return clean_title_list

[nltk_data] Downloading package stopwords to /Users/anson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/anson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/anson/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
# Prepare model inputs of bag of words approach

# Create lists of cleaned train, test, and all titles
train_clean_title_list = clean_df_title_col(train_df['Title'])
test_clean_title_list = clean_df_title_col(test_df['Title'])
all_clean_title_list = train_clean_title_list + test_clean_title_list

# Create count vectorizer and bag of words
vectorizer = CountVectorizer(max_features=3000)
BOW = vectorizer.fit_transform(all_clean_title_list)

# Create list of all labels
all_label_list = train_df["Label"].tolist() + test_df["Label"].tolist()

In [7]:
# Run model

# Re-split data due to bag of words
X_train, X_test, y_train, y_test = train_test_split(BOW, all_label_list)

# Create and fit classifier
#model = SVC(kernel='linear', C=0.1)
#model.fit(X_train,y_train)

# Save / Load classifier
#dump(model, 'Model/svm_model.joblib')
model = load('Model/svm_model.joblib')

# Make predictions on test data
y_pred = model.predict(X_test)

# Calculate accuracy and other metrics
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print results
print("Classifier Accuracy:", accuracy)
print("Classification Report:")
print(report)

Classifier Accuracy: 0.8941261783901377
Classification Report:
              precision    recall  f1-score   support

           1       0.78      0.93      0.85       347
           2       0.98      0.96      0.97       350
           3       0.95      0.92      0.94       305
           4       0.89      0.91      0.90       163
           5       0.97      0.82      0.89       133
           6       0.80      0.41      0.54        81

    accuracy                           0.89      1379
   macro avg       0.90      0.83      0.85      1379
weighted avg       0.90      0.89      0.89      1379

